In [ ]:
from __future__ import division, print_function, absolute_import

import tensorflow as tf

import matplotlib.pyplot as plt            #Display the images for verification
import numpy as np                         #Mathematical and arrays library
from scipy import misc                     #Image Importing module
from skimage.transform import resize       #Resizing image module
from sklearn.utils import shuffle          #Shuffle in random order module
import glob                                #Iterating through all pictures library

In [ ]:
# Training Parameters
learning_rate = 0.001
num_steps = 500
batch_size = 128
display_step = 10

# Network Parameters
num_input = 784 # MNIST data input (img shape: 28*28)
num_classes = 3 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

# tf Graph input
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)

In [ ]:
# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')


# Create model
def conv_net(x, weights, biases, dropout):
    # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
    # Reshape to match picture format [Height x Width x Channel]
    # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [ ]:
#Creating initial array and resizing
f_image_list = []
img_count = 0
path = "/Users/hannansaleemi/Desktop/SDC/Jupyter Notebook Examples/cardataset/training_set/Forward/"
for filename in glob.glob(path+'*.png'):
    img_count += 1
    image = misc.imread(path+'trainframe ('+str(img_count)+').png')
    image_resized = resize(image, (28,28), mode='reflect')
    f_image_list.append(image_resized)
    
#Converting to greyscale
f_grey = np.zeros((3000, image_resized.shape[0], image_resized.shape[1]))

for image_num in range(len(f_image_list)):
    for rownum in range(len(f_image_list[image_num])):
        for colnum in range(len(f_image_list[image_num][rownum])):
            f_grey[image_num][rownum][colnum] = weightedAverage(f_image_list[image_num][rownum][colnum])
    #Output Logging
    if image_num % 100 == 0:
        print("Finished converting image #", (image_num))
        
#Just checking the shape of the greyscale array
f_grey.shape

#Reshaping
f_images = f_grey.reshape(3000,784)

#Creating Labels
f_labels = np.zeros(shape=(3000,3))

#Adding 3000 forward labels [1,0,0]
for i in range(3000):
    f_labels[i][0] = float(1)
    
#Shuffling the images
f_images, f_labels = shuffle(f_images, f_labels, random_state = 4000)



#Creating initial array and resizing
l_image_list = []
img_count = 0
path = "/Users/hannansaleemi/Desktop/SDC/Jupyter Notebook Examples/cardataset/training_set/Left/"
for filename in glob.glob(path+'*.png'):
    img_count += 1
    image = misc.imread(path+'trainframe ('+str(img_count)+').png')
    image_resized = resize(image, (28,28), mode='reflect')
    l_image_list.append(image_resized)
    

#Converting to greyscale
l_grey = np.zeros((3000, image_resized.shape[0], image_resized.shape[1]))

for image_num in range(len(l_image_list)):
    for rownum in range(len(l_image_list[image_num])):
        for colnum in range(len(l_image_list[image_num][rownum])):
            l_grey[image_num][rownum][colnum] = weightedAverage(l_image_list[image_num][rownum][colnum])
    #Output Logging
    if image_num % 100 == 0:
        print("Finished converting image #", (image_num))
        
l_images = l_grey.reshape(3000,784)

l_labels = np.zeros(shape=(3000,3))

#Adding 3000 forward labels [1,0,0]
for i in range(3000):
    l_labels[i][1] = float(1)
    
l_images, l_labels = shuffle(l_images, l_labels, random_state = 4000)



#Creating initial array and resizing
r_image_list = []
img_count = 0
path = "/Users/hannansaleemi/Desktop/SDC/Jupyter Notebook Examples/cardataset/training_set/Right/"
for filename in glob.glob(path+'*.png'):
    img_count += 1
    image = misc.imread(path+'trainframe ('+str(img_count)+').png')
    image_resized = resize(image, (28,28), mode='reflect')
    r_image_list.append(image_resized)
    
    
#Converting to greyscale
r_grey = np.zeros((3000, image_resized.shape[0], image_resized.shape[1]))

for image_num in range(len(r_image_list)):
    for rownum in range(len(r_image_list[image_num])):
        for colnum in range(len(r_image_list[image_num][rownum])):
            r_grey[image_num][rownum][colnum] = weightedAverage(r_image_list[image_num][rownum][colnum])
    #Output Logging
    if image_num % 100 == 0:
        print("Finished converting image #", (image_num))
        
        
#Reshaping
r_images = r_grey.reshape(3000,784)

#Creating Labels
r_labels = np.zeros(shape=(3000,3))

#Adding 3000 forward labels [1,0,0]
for i in range(3000):
    r_labels[i][2] = float(1)
    
#Shuffling the images
r_images, r_labels = shuffle(r_images, r_labels, random_state = 4000)

all_images_array = np.concatenate((f_images, l_images, r_images), axis=0)
all_labels_array = np.concatenate((f_labels, l_labels, r_labels), axis=0)

#Shuffling the images
all_images_array, all_labels_array = shuffle(all_images_array, all_labels_array, random_state = 9000)

In [ ]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, num_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

# Construct model
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)


# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()